In [1]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

# Get Gaia catalog of the region

Use this catalog + ds9 to quickly see which image has the best-calibrated coordinates (usually JWST F090W is pretty good).

In [12]:
galaxy_name = 'N2525'
center_coord = SkyCoord.from_name(galaxy_name)
width = u.Quantity(10, u.arcmin)
height = u.Quantity(10, u.arcmin)

In [13]:
query = """SELECT 
TOP 3000
source_id, ra, dec, pm, parallax, phot_g_mean_mag, bp_rp, pmra_error, pmdec_error, parallax_error
FROM gaiadr3.gaia_source
WHERE pm < 100 
AND parallax > 0
AND parallax < 100
AND parallax_error < 4
AND pmra_error < 4
AND pmdec_error < 4
AND phot_g_mean_mag BETWEEN 10 AND 21
"""
query_loc = f'AND ra BETWEEN {center_coord.ra.deg-width.to(u.deg).value} AND {center_coord.ra.deg+width.to(u.deg).value}\n'+\
            f'AND dec BETWEEN {center_coord.dec.deg-height.to(u.deg).value} AND {center_coord.dec.deg+height.to(u.deg).value}'
            
job = Gaia.launch_job(query+query_loc)

r = job.get_results()
df = r.to_pandas()
df

,SOURCE_ID,ra,dec,pm,parallax,phot_g_mean_mag,bp_rp,pmra_error,pmdec_error,parallax_error
0,3037744782651425920,121.242100,-11.260503,6.858300,0.471396,20.405565,2.354418,0.617413,0.501258,0.734251
1,3037744782653540992,121.242271,-11.262919,7.180786,1.744496,11.149982,0.483756,0.027189,0.019702,0.025805
2,3037741140522996352,121.242275,-11.304987,1.381465,0.246782,19.714916,0.659145,0.362005,0.306619,0.457995
3,3037733787537314688,121.242364,-11.414719,4.420003,0.273641,15.194893,0.488214,0.026331,0.020676,0.027365
4,3037733959331380864,121.242462,-11.394091,24.282324,0.139532,19.378407,2.297344,0.340940,0.259173,0.384395
...,...,...,...,...,...,...,...,...,...,...
2570,3037820167919139584,121.574469,-11.327758,4.621075,0.359743,18.415007,0.991396,0.149410,0.093489,0.147602
2571,3037725914857097856,121.574607,-11.395815,2.414740,0.136635,19.024700,1.135693,0.250658,0.143003,0.247229
2572,5727871568241869056,121.574658,-11.561284,0.993171,0.007955,19.132429,0.763783,0.224900,0.162230,0.243800
2573,3037724952784352512,121.575049,-11.433328,28.297544,2.095191,19.320066,2.709276,0.294489,0.174947,0.305651


In [14]:
with open(f'/Volumes/S-Express/SH0ES_reprojected/{galaxy_name}/gaia_catalog.reg','w') as f:
    for i in df.index.values:
        ra,dec = df.loc[i,['ra','dec']]
        coord = SkyCoord(ra=ra*u.deg,dec=dec*u.deg)
        coord_str = coord.to_string('hmsdms',sep=':',precision=3)
        f.write(f'Circle({coord_str},1")\n')
        